In [58]:
import json
import sys
import glob
import torch
sys.path.append('../')
import os
from transformers import *
from frameBERT.src import utils
from frameBERT.src import dataio
from frameBERT.src import eval_fn
from frameBERT import frame_parser
from frameBERT.src.modeling import BertForJointShallowSemanticParsing
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from torch import nn
from torch.optim import Adam
from tqdm import tqdm, trange

from pprint import pprint

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
if device != "cpu":
    torch.cuda.set_device(0)
import pickle

import numpy as np
import random
np.random.seed(0)   
random.seed(0)

from torch import autograd
torch.cuda.empty_cache()

In [59]:
# from frameBERT.src import dataio
# srl = 'framenet'
# language = 'en'
# fnversion = '1.5'
# trn, dev, tst = dataio.load_data(language=language, fnversion=fnversion)
# print(trn[0])
# d = trn+dev+tst

# with open('./data/frame_coreFE_list_v1.5.json','r') as f:
#     fc = json.load(f)
    
# n = 0
# for i in d:
#     lu, frame = get_lu_frame(i)
#     luidx = lu2idx[lu]
#     frameidx = frame2idx[frame]
#     fes, bio_fes = get_fes(i)
    
#     corefe = fc[frame]
#     for fe in fes:
#         if fe in corefe:
#             n +=1
# print(n)

In [60]:
# from frameBERT.src import eval_fn
# eval_fn.weighting()

In [61]:
srl = 'framenet'
language = 'en'
fnversion = '1.5'

In [62]:
# bert_io = utils.for_BERT(mode='train', language=language, masking=True, fnversion=fnversion)
trn, dev, tst = dataio.load_data(language=language, exem=True, fnversion=fnversion, info=True)
print(trn[0])

[['12-year-old', 'boy', 'reports', 'seeing', 'man', 'launch', 'rubber', 'boat', 'from', 'car', 'parked', 'the', '<tgt>', 'harbor', '</tgt>', '.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'harbor.n', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Locale_by_use', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'B-Locale', 'X', 'O']]


In [63]:
d = trn+dev+tst

In [65]:
for i in d:
    print(i)
    break
print(len(d))

[['12-year-old', 'boy', 'reports', 'seeing', 'man', 'launch', 'rubber', 'boat', 'from', 'car', 'parked', 'the', '<tgt>', 'harbor', '</tgt>', '.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'harbor.n', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Locale_by_use', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'B-Locale', 'X', 'O']]
177269


In [73]:
fes = []
dir_path = './koreanframenet/resource/info/fn1.5_'
fname = dir_path+'fe2idx.json'
with open(fname, 'r') as f:
    old_fe2idx = json.load(f)
    
for fe in old_fe2idx:
    fes.append(fe)
    
for i in d:
    for arg in i[-1]:
        if arg != 'O':
            if arg != 'X':
                fe = arg[2:]
                fes.append(fe)
                
fes = list(set(fes))
fes.sort()
print(len(fes))
# print(fes)

fe2idx = {}
for fe in fes:
    fe2idx[fe] = len(fe2idx)
with open(fname, 'w') as f:
    json.dump(fe2idx, f, ensure_ascii=False, indent=4)

1068


In [74]:
dir_path = './koreanframenet/resource/info/fn1.5_'
fname = dir_path+'bio_fe2idx.json'
fname = dir_path+'fe2idx.json'
with open(fname, 'r') as f:
    fe2idx = json.load(f)
# for i in fe2idx:
#     print(i)
#     break

bio_fes = []
for i in d:
    bio_fes += i[-1]
    
bio_fes = list(set(bio_fes))
fes = []
for i in bio_fes:
    if i != 'O':
        if i != 'X':
            fe = i[2:]
            fes.append(fe)
    
fes = list(set(fes))

for i in fes:
    idx = fe2idx[i]
    
bio_fe2idx = {}
bio_fe2idx['O'] = len(bio_fe2idx)
bio_fe2idx['X'] = len(bio_fe2idx)

for i in fe2idx:
    bio_fe = 'B-'+i
    bio_fe2idx[bio_fe] = len(bio_fe2idx)
    bio_fe = 'I-'+i
    bio_fe2idx[bio_fe] = len(bio_fe2idx)
    
with open('./koreanframenet/resource/info/fn1.5_bio_fe2idx.json','w') as f:
    json.dump(bio_fe2idx, f, ensure_ascii=False, indent=4)

In [75]:
def get_lu_frame(conll):
    lus = conll[1]
    frames = conll[2]
    for i in range(len(lus)):
        lu_i = lus[i]
        frame_i = frames[i]
        
        if lu_i != '_':
            lu = lu_i
            frame = frame_i
    return lu, frame

def get_fes(conll):
    bio_fes = list(set(conll[-1]))
    fes = []
    for i in bio_fes:
        if i == 'O':
            fe = i
        elif i == 'X':
            fe = i
        else:
            fe = i[2:]
        fes.append(fe)

    return fes, bio_fes 

In [78]:
with open('./koreanframenet/resource/info/fn1.5_lu2idx.json', 'r') as f:
    lu2idx = json.load(f)
    
lus = []
for lu in lu2idx:
    lus.append(lu)

for i in d:
    lu, frame = get_lu_frame(i)
    lus.append(lu)
    
lus = list(set(lus))
lus.sort()

print(len(lus))

lu2idx = {}
for lu in lus:
    lu2idx[lu] = len(lu2idx)
with open('./koreanframenet/resource/info/fn1.5_lu2idx.json', 'w') as f:
    json.dump(lu2idx, f, ensure_ascii=False, indent=4)

7762


In [83]:
with open('./koreanframenet/resource/info/fn1.5_frame2idx.json', 'r') as f:
    frame2idx = json.load(f)

frames = []
for fr in frame2idx:
    frames.append(fr)
    
for i in d:
    lu, frame = get_lu_frame(i)
    frames.append(frame)
    
frames = list(set(frames))
frames.sort()

frame2idx = {}
for fr in frames:
    frame2idx[fr] = len(frame2idx)
    
with open('./koreanframenet/resource/info/fn1.5_frame2idx.json', 'w') as f:
    json.dump(frame2idx, f, ensure_ascii=False, indent=4)
    
print(len(frame2idx))

876


In [84]:
with open('./koreanframenet/resource/info/fn1.5_lu2idx.json', 'r') as f:
    lu2idx = json.load(f)
    
with open('./koreanframenet/resource/info/fn1.5_frame2idx.json', 'r') as f:
    frame2idx = json.load(f)
    
lufrmap = {}
for i in d:
    lu, frame = get_lu_frame(i)
    luidx = lu2idx[lu]
    frameidx = frame2idx[frame]
    
    if luidx in lufrmap:
        frame_candis = lufrmap[luidx]
    else:
        frame_candis = []
        
    frame_candis.append(frameidx)
    frame_candis = list(set(frame_candis))
    lufrmap[luidx] = frame_candis
    
with open('./koreanframenet/resource/info/fn1.5_lufrmap.json', 'w') as f:
    json.dump(lufrmap, f, ensure_ascii=False, indent=4)

In [85]:
# lufrmap check

with open('./koreanframenet/resource/info/fn1.5_lufrmap.json', 'r') as f:
    lufrmap = json.load(f)
    
with open('./koreanframenet/resource/info/fn1.5_lu2idx.json', 'r') as f:
    lu2idx = json.load(f)
    
with open('./koreanframenet/resource/info/fn1.5_frame2idx.json', 'r') as f:
    frame2idx = json.load(f)
    
# lufrmap = {}
for i in d:
    lu, frame = get_lu_frame(i)
    luidx = lu2idx[lu]
    frameidx = frame2idx[frame]
    frame_candis = lufrmap[str(luidx)]
    if frameidx not in frame_candis:
        print(lu)

In [87]:
# frargmap

with open('./koreanframenet/resource/info/fn1.5_lufrmap.json', 'r') as f:
    lufrmap = json.load(f)
    
with open('./koreanframenet/resource/info/fn1.5_lu2idx.json', 'r') as f:
    lu2idx = json.load(f)
    
with open('./koreanframenet/resource/info/fn1.5_frame2idx.json', 'r') as f:
    frame2idx = json.load(f)
    
with open('./koreanframenet/resource/info/fn1.5_fe2idx.json', 'r') as f:
    fe2idx = json.load(f)
    
with open('./koreanframenet/resource/info/fn1.5_bio_fe2idx.json', 'r') as f:
    bio_fe2idx = json.load(f)
    
# with open('./koreanframenet/resource/info/fn1.5_frame2idx.json', 'r') as f:
#     frargmap = json.load(f)
    
# lufrmap = {}

frargmap, bio_frargmap = {},{}
for i in d:
    lu, frame = get_lu_frame(i)
    luidx = lu2idx[lu]
    frameidx = frame2idx[frame]
    fes, bio_fes = get_fes(i)
    
    if frameidx in frargmap:
        fe_candis = frargmap[frameidx]
    else:
        fe_candis = []
    if frameidx in bio_frargmap:
        bio_fe_candis = bio_frargmap[frameidx]
    else:
        bio_fe_candis = []
        
    for fe in fes:
        if fe != 'O':
            if fe != 'X':
                fe_idx = fe2idx[fe]
                fe_candis.append(fe_idx)
    for bio_fe in bio_fes:
        bio_fe_idx = bio_fe2idx[bio_fe]
        bio_fe_candis.append(bio_fe_idx)
        
    fe_candis = list(set(fe_candis))
    bio_fe_candis = list(set(bio_fe_candis))
    
    frargmap[frameidx] = fe_candis
    bio_frargmap[frameidx] = bio_fe_candis
    
with open('./koreanframenet/resource/info/fn1.5_frargmap.json', 'w') as f:
    json.dump(frargmap, f, ensure_ascii=False, indent=4)
    
with open('./koreanframenet/resource/info/fn1.5_bio_frargmap.json', 'w') as f:
    json.dump(bio_frargmap, f, ensure_ascii=False, indent=4)